<a href="https://colab.research.google.com/github/RafaelCaballero/APD/blob/main/codigo/web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table>
<tr>
<td><img src="https://raw.githubusercontent.com/RafaelCaballero/APD/refs/heads/main/img/logoAPD.png" width="150"></td>
<td><table><tr><td><h1>Carga de datos desde Web</h1></td></tr>
           <tr><td><h3>Rafael Caballero Roldán</h3></td></tr></table></td>
<td><img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTsPjCdm67xYS9AM7-dXQ46O23vaexAhnVJaQ&s" width="105"></td>
</tr>
</table>





### Un poquito de Web Scraping

Cuando los datos no son fáciles de obtener de una página web debemos analizar directemente el código HTML de la práctica. Para esto podemos usar, entre otras, dos bibliotecas:

- BeautifulSoup: el estándar para manejar directamente código HTML. Permite navegar por los elementos de la página de forma sencilla
- Selenium: cuando la página requiere interactividad, y se quiere automatizar la pulsación de botones, selección de listas desplegables, etc.


In [ ]:
modules = ["numpy","matplotlib","numpy","tqdm","yfinance", "html5lib","xlrd", "openpyxl","lxml","beautifulsoup4","Selenium"]

import sys
import os.path
from subprocess import check_call
import importlib
import os

def instala(modules):
    print("Instalando módulos")
    for m in modules:
        # para el import quitamos [...] y ==...
        p = m.find("[")
        mi = m if p==-1 else m[:p]
        p = mi.find("==")
        mi = mi if p==-1 else mi[:p]
        torch_loader = importlib.util.find_spec(mi)
        if torch_loader is not None:
            print(m," encontrado")
        else:
            print(m," No encontrado, instalando...",end="")
            try:
                r = check_call([sys.executable, "-m", "pip", "install", "--user", m])
                print("¡hecho!")
            except:
                print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")

    print("¡Terminado!")

instala(modules)

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

url = "https://www.worldometers.info/es/geografia/paises-del-mundo/"


# error, no hay tablas, por eso usamos BeautifulSoup.
#dfs = pd.read_html(url)  # error no tables found o forbidden
#print(len(dfs))

r = requests.get(url)
print("requests encoding:", r.encoding, r.apparent_encoding)
r.encoding = "utf-8"  # lo forzamos


soup = BeautifulSoup(r.text, 'html.parser')  # le pasamos el texto en HTML para que lo analice


El paso anterior ha analizado el texto y extraído sus componentes. Ahora podemos escribir, por ejemplo:

In [ ]:
print(soup.head.title)

In [ ]:
print(soup.head.title.text)

Esta forma de "navegar" la página de arriba a abajo puede resultar muy tediosa y además solo permite llegar a algunos elementos, es la llamada "dot navegation" que usaremos cuando ya estemos cerca de la información que queremos

In [ ]:
for e in soup.body.children:
    print(e.name)



Es importante determinar si la información que vemos está en la página descargada; no siempre es así

In [ ]:
"India" in soup.text

Para buscar con más libertad tendremos otras funciones como 'find' y 'find_all' que encuentran, respectivamente, la primera aparición o todas las apariciones de un tag:

In [ ]:
cabecera = soup.find('h1')
cabecera

In [ ]:
cabeceras = soup.find_all('h2')
len(cabeceras)

In [ ]:
cabeceras[0]

Ahora vamos a buscar la información que nos interesa. Tras inspeccionar la página en google Chrome (por ejemplo), vemos que parece que cada fila viene en un elemento de tipo `div` con atributo  class="table-row". La función `select`devuelve todos los elementos que cumplen esto (nótese el . antes del nombre)

In [ ]:
tables = soup.find_all("table")
print(len(tables))

In [ ]:
tables[0].text

In [ ]:
tables[1].text

In [ ]:
table = tables[0]

Ahora navegamos a partir de table, eso hace que sea más rápido y más fácil no equivocarse. Recorrido por filas de la tabla

In [ ]:
trs = table.find_all("tr")
for tr in trs:
    print(tr.text)

Sobra la primera fila. Además aprovechamos para sacar los componentes tds

In [ ]:
trs = table.find_all("tr")
for tr in trs[1:]:
    tds = tr.find_all("td")
    print(len(tds))


In [ ]:
trs = table.find_all("tr")
lista = []
for tr in trs[1:]:
    tds = tr.find_all("td")
    # posición
    fila = [tds[1].text.strip(),int(tds[2].text.strip().replace(".","")),tds[3].text.strip()]
    lista.append(fila)

lista

Convertimos a DataFrame

In [ ]:
import pandas as pd

df = pd.DataFrame(lista,columns=["Nombre","Habitantes","Continente"])
df

In [ ]:
df[df.Nombre=="España"]

Todo junto:

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pandas as pd

url = "https://www.worldometers.info/es/geografia/paises-del-mundo/"



r = requests.get(url)
r.encoding = "utf-8"  # lo forzamos
soup = BeautifulSoup(r.text, 'html.parser')

tables = soup.find_all("table")
table = tables[0]

trs = table.find_all("tr")
lista = []
for tr in trs[1:]:
    tds = tr.find_all("td")
    # posición
    fila = [tds[1].text.strip(),int(tds[2].text.strip().replace(".","")),tds[3].text.strip()]
    lista.append(fila)



df.to_csv("paises.csv",index=False)
df = pd.DataFrame(lista,columns=["Nombre","Habitantes","Continente"])
df
